1. Install and Import Dependencies

In [3]:
# MPS acceleration is available on MacOS 12.3!+
!pip install torch torchvision torchaudio
#if kernel dies when importing torch run the following uninstall&install
!pip3 uninstall -y torch torchvision
!pip3 install torch torchvision

Found existing installation: torch 2.0.1
Uninstalling torch-2.0.1:
  Successfully uninstalled torch-2.0.1
Found existing installation: torchvision 0.15.2
Uninstalling torchvision-0.15.2:
  Successfully uninstalled torchvision-0.15.2
  Using cached torch-2.0.1-cp39-none-macosx_10_9_x86_64.whl (143.4 MB)
  Using cached torchvision-0.15.2-cp39-cp39-macosx_10_9_x86_64.whl (1.5 MB)


In [4]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
from bs4 import BeautifulSoup
import re
import torch

2. Instantiate Model

In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

3. Encode and Calculate Sentiment

In [8]:
tokens = tokenizer.encode('I hated this, absolutely the worst', return_tensors='pt')

In [9]:
tokens

tensor([[  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
           102]])

In [10]:
result = model(tokens)

In [11]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.8750,  1.7880, -0.8356, -3.0027, -2.0727]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [12]:
int(torch.argmax(result.logits))+1

1

4. Collect Reviews

In [16]:
r = requests.get('https://www.yelp.com/biz/the-butterfly-and-the-pig-glasgow')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [17]:
reviews

['Amazing service and food. Highly recommend. Accommodated a large party of 7 with no problems. Waitress was lovely!',
 "Such a cute and quaint place for afternoon tea. We found the place on Yelp and wanted to give it a try. Didn't have reservations but there was a good amount of tables available downstairs. Found out later that they had more seating upstairs. Staff was cute sweet and friendly. The whole area where we were seated was quite shabby chic with all the vintage and mismatched dishes and cutlery. Wooden table with lace tablecloth. We came for afternoon tea which was quite affordable. They also have upgrade to bubbly if you wanted something extra. The three tier offered sandwiches, cakes, scones, salmon on top of crackers and served with chips. Everything was great. Presentation was beautiful. Tea was unlimited I believe which is nice a/ some places charge for refills. There's also a la carte dishes as well. Overall, cute spot for tea.",
 'I had bookmarked this place for dinne

5. Load Reviews into DataFrame and Score

In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [22]:
df.head()

,review
0,Amazing service and food. Highly recommend. Ac...
1,Such a cute and quaint place for afternoon tea...
2,"I had bookmarked this place for dinner, but wh..."
3,A very quaint but fun and lively atmosphere! T...
4,Fantastic pub. Best fish and chips. Large port...


In [23]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [26]:
sentiment_score(df['review'].iloc[0])

5

In [27]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
#NLP can't parse more than 512 token at one time

In [28]:
df

,review,sentiment
0,Amazing service and food. Highly recommend. Ac...,5
1,Such a cute and quaint place for afternoon tea...,4
2,"I had bookmarked this place for dinner, but wh...",4
3,A very quaint but fun and lively atmosphere! T...,4
4,Fantastic pub. Best fish and chips. Large port...,5
5,Despite everyone saying the menu was confusing...,4
6,Wonderful place to grab some grub and enjoy a ...,5
7,The food is amazing food choices and drink a w...,5
8,It was a little difficult to find this place b...,5
9,"Really enjoyed the cool venue and food, the se...",4
